In [84]:
from collections import Counter
from scipy.stats import beta, bernoulli
import numpy as np

In [138]:
# Context-Free Bandit implementation
class TS():
    def __init__(self, arm_ids) :
        self.arm_ids = arm_ids
        self.counts = {}
        self.values = {}
        self.beta_params = {}
        for idx, arm_id in enumerate(arm_ids):
            self.counts[arm_id] = 0
            self.values[arm_id] = 0.0
            self.beta_params[arm_id] = {}
            self.beta_params[arm_id]['success'] = 1
            self.beta_params[arm_id]['failure'] = 1
            
    def select_arm ( self ):
        sample_success_rates = []
        for idx, arm_id in enumerate(self.arm_ids):
            sample_success_rates.append( 
                (
                    arm_id,
                    beta.rvs( tsObj.beta_params[arm_id]['success'], tsObj.beta_params[arm_id]['failure'])
                )
            )
        return max(sample_success_rates,key=lambda x:x[1])
    
    def update( self, chosen_arm, reward ):
        self.counts[chosen_arm] += 1
        n = self.counts[chosen_arm]
        
        value = self.values[ chosen_arm ]
        new_value = ( (n-1)* value + reward ) / float(n)
        self.values[chosen_arm] = new_value        

In [140]:
true_reward_rates = {
    1: 0.45,
    2: 0.47,
    3: 0.5
}

noOfTrails = 5000
tsObj = TS( true_reward_rates.keys() ) 

tCnter = Counter()

for t in range(noOfTrails):

    chosen_arm, sample_reward_rate = tsObj.select_arm()
    tCnter[chosen_arm]  += 1
    bernoulli_reward = bernoulli.rvs(true_reward_rates[chosen_arm])
    if bernoulli_reward == 1:
        tsObj.beta_params[chosen_arm]['success'] += 1
    else:
        tsObj.beta_params[chosen_arm]['failure'] += 1
    
tsObj.beta_params  
tCnter

Counter({3: 4533, 1: 43, 2: 424})